# Liquid Liquid Equilibrium (LLE)

Phase stability plays a key role during equilibrium computation when dealing with more than two liquid phases. For this purpose the following modified multiphase Rachford-Rice mass balance has been proposed by [Gupta et al.](https://www.sciencedirect.com/science/article/pii/037838129180021M):


$$ \sum_{i=1}^c \frac{z_i (K_{ik} \exp{\theta_k}-1)}{1+ \sum\limits^{\pi}_{\substack{j=1 \\ j \neq r}}{\psi_j (K_{ij}} \exp{\theta_j} -1)} = 0 \qquad k = 1,..., \pi,  k \neq r $$

Subject to:

$$ \psi_k \theta_k = 0 $$

In this system of equations, $z_i$ represents the global composition of the component $i$,  $ K_{ij} = x_{ij}/x_{ir} = \hat{\phi}_{ir}/\hat{\phi}_{ij} $ is the constant equilibrium of component $i$ in phase $j$ respect to the reference phase $r$, and $\psi_j$ and $\theta_j$ are the phase fraction and stability variable of the phase $j$.  

The solution strategy is similar to the classic isothermal isobaric two-phase flash. First, a reference phase must be selected, this phase is considered stable during the procedure. In an inner loop, the system of equations is solved using multidimensional Newton's method for phase fractions and stability variables and then compositions are updated in an outer loop using accelerated successive substitution (ASS).  Once the algorithm has converged, the stability variable gives information about the phase. If it takes a value of zero the phase is stable and if it is positive the phase is not.  The proposed successive substitution method can be slow, if that is the case the algorithm attempts to minimize Gibbs Free energy of the system. This procedure also ensures stable solutions and is solved using SciPy's functions.

$$ min \, {G} = \sum_{k=1}^\pi \sum_{i=1}^c F_{ik} \ln \hat{f}_{ik}  $$

This notebook shows the solution of liquid-liquid equilibrium using the ``lle`` function. This function incorporates the algorithm described above. To start, the required functions are imported.

In [1]:
import numpy as np
from sgtpy import component, mixture, saftvrmie
from sgtpy.equilibrium import lle, lle_init

---
### Binary example

The LLE calculation for the mixture of water and butanol will be exemplified. First the mixture and its interaction parameters are set up. 

In [2]:
# creating pure components
water = component('water', ms = 1.7311, sigma = 2.4539 , eps = 110.85,
                    lambda_r = 8.308, lambda_a = 6., eAB = 1991.07, rcAB = 0.5624,
                    rdAB = 0.4, sites = [0,2,2], cii = 1.5371939421515458e-20)

butanol = component('butanol2C', ms = 1.9651, sigma = 4.1077 , eps = 277.892,
                    lambda_r = 10.6689, lambda_a = 6., eAB = 3300.0, rcAB = 0.2615,
                    rdAB = 0.4, sites = [1,0,1], npol = 1.45, mupol = 1.6609,
                    cii  = 1.5018715324070352e-19)
mix = mixture(water, butanol)
# or
mix = water + butanol

# optimized from experimental LLE
kij, lij = np.array([-0.00736075, -0.00737153])
Kij = np.array([[0, kij], [kij, 0]])
Lij = np.array([[0., lij], [lij, 0]])

# setting interactions corrections
mix.kij_saft(Kij)
mix.lij_saft(Lij)
# creating eos model
eos_bin = saftvrmie(mix)

Then the LLE is computed at given global composition (``z``), temperature (``T``) and pressure (``P``). The function ``lle``require initial guesses of the compositions of the phases. You can use the ``lle_init`` function to obtain initial guesses for the phase compositions.

In [3]:
T = 298.15 # K
P = 1.01325e5 # Pa

# global composition
z = np.array([0.8,0.2])
# initial guesses for liquids compositions
x0 = np.array([0.9, 0.1])
w0 = np.array([0.6, 0.4])

# LLE is performed as a flash that search stable phases
lle(x0, w0, z, T, P, eos_bin, full_output=False)

(array([0.96021656, 0.03978344]),
 array([0.53380265, 0.46619735]),
 0.37573014340305205)

You can also supply initial guesses for the phase volumes (``v0``) or non-bonded association site fractions (``Xass0``), which can come from a previous calculation using the ``full_output=True`` option.

In [4]:
T = 298.15 # K
P = 1.01325e5 # Pa

# global composition
z = np.array([0.8,0.2])
# initial guesses for liquids compositions
x0 = np.array([0.9, 0.1])
w0 = np.array([0.6, 0.4])
sol_lle = lle(x0, w0, z, T, P, eos_bin, full_output=True)
sol_lle

           T: 298.15
           P: 101325.0
 error_outer: 7.0326050241244155e-09
 error_inner: 6.790912850114029e-09
        iter: 16
        beta: array([0.62426986, 0.37573014])
       tetha: array([0.])
           X: array([[0.96021656, 0.03978344],
       [0.53380265, 0.46619735]])
           v: [2.03145764150821e-05, 5.135704742474372e-05]
        Xass: [array([0.05681864, 0.07391832, 0.01687673, 0.03815106]), array([0.06370272, 0.19811159, 0.02977199, 0.1168081 ])]
      states: ['L', 'L']

In [5]:
T = 298.15 # K
P = 1.01325e5 # Pa

# global composition
z = np.array([0.8,0.2])
# initial guesses for liquids compositions
x0 = np.array([0.9, 0.1])
w0 = np.array([0.6, 0.4])
v0 = sol_lle.v
Xass0 = sol_lle.Xass
# LLE supplying initial guess for volumes and non-bonded association sites fractions
lle(x0, w0, z, T, P, eos_bin, v0=v0, Xass0=Xass0, full_output=True)

           T: 298.15
           P: 101325.0
 error_outer: 7.032604997925568e-09
 error_inner: 6.790912850114029e-09
        iter: 16
        beta: array([0.62426986, 0.37573014])
       tetha: array([0.])
           X: array([[0.96021656, 0.03978344],
       [0.53380265, 0.46619735]])
           v: [2.0314576415081104e-05, 5.135704742474714e-05]
        Xass: [array([0.05681864, 0.07391832, 0.01687673, 0.03815106]), array([0.06370272, 0.19811159, 0.02977199, 0.1168081 ])]
      states: ['L', 'L']

Initial guesses for the phase compositions can be obtained using the ``lle_init`` function. This is shown below.

In [6]:
T = 298.15 # K
P = 1.01325e5 # Pa
# global composition
z = np.array([0.8,0.2])

lle_init(z, T, P, eos_bin)

(array([0.9598314, 0.0401686]), array([0.53075737, 0.46924263]))

---
### Ternary example

The ``lle`` function is not restricted to binary mixtures. Its use for a ternary mixture is shown below:

In [7]:
water = component('water', ms = 1.7311, sigma = 2.4539 , eps = 110.85,
                    lambda_r = 8.308, lambda_a = 6.,  eAB = 1991.07, rcAB = 0.5624,
                    rdAB = 0.4, sites = [0,2,2], cii = 1.5371939421515455e-20)

butanol = component('butanol2C', ms = 1.9651, sigma = 4.1077 , eps = 277.892,
                    lambda_r = 10.6689, lambda_a = 6., eAB = 3300.0, rcAB = 0.2615,
                    rdAB = 0.4, sites = [1,0,1], npol = 1.45, mupol = 1.6609,
                    cii  = 1.5018715324070352e-19)

mtbe = component('mtbe', ms =2.17847383,  sigma=  4.19140014, eps =  306.52083841,
                 lambda_r = 14.74135198, lambda_a = 6.0, npol = 2.95094686,  
                 mupol = 1.3611, sites = [0,0,1], cii =3.5779968517655445e-19 )

mix = mixture(water, butanol)
mix.add_component(mtbe)
# or
mix = water + butanol + mtbe

#butanol water
k12, l12 = np.array([-0.00736075, -0.00737153])

#mtbe butanol
k23 = -0.0029995
l23 = 0.
rc23 =  1.90982649

#mtbe water
k13 = -0.07331438
l13 = 0.
rc13 = 2.84367922

# setting up interaction corrections
Kij = np.array([[0., k12, k13], [k12, 0., k23], [k13, k23, 0.]])
Lij = np.array([[0., l12, l13], [l12, 0., l23], [l13, l23, 0.]])
mix.kij_saft(Kij)
mix.lij_saft(Lij)
eos = saftvrmie(mix)

# setting up induced association
#mtbe water
eos.eABij[0,2] = water.eAB / 2
eos.eABij[2,0] = water.eAB / 2
eos.rcij[0,2] = rc13 * 1e-10
eos.rcij[2,0] = rc13 * 1e-10
#mtbe butanol
eos.eABij[2,1] = butanol.eAB / 2
eos.eABij[1,2] = butanol.eAB / 2
eos.rcij[2,1] = rc23 * 1e-10
eos.rcij[1,2] = rc23 * 1e-10

T = 340. #K
P = 1.01325e5 # Pa
# global composition
z = np.array([0.5, 0.3, 0.2])
# initial guesses
x0 = np.array([0.9, 0.05, 0.05])
w0 = np.array([0.45, 0.45, 0.1])
lle(x0, w0, z, T, P, eos, full_output = True)

           T: 340.0
           P: 101325.0
 error_outer: 7.87784767529773e-09
 error_inner: 1.257728144083193e-10
        iter: 12
        beta: array([0.21178723, 0.78821277])
       tetha: array([0.])
           X: array([[0.96977166, 0.02411117, 0.00611718],
       [0.37377566, 0.37412939, 0.25209495]])
           v: [2.0423180772183556e-05, 7.29029606060134e-05]
        Xass: [array([0.09206316, 0.10554198, 0.03316298, 0.06878314, 0.1152663 ]), array([0.12441032, 0.37577807, 0.08464571, 0.29547323, 0.45318969])]
      states: ['L', 'L']

---
For further information about the lle function check out the documentation running: ``lle?``